In [ ]:
%%sh
cd ~/data-analyses/rt_segment_speeds
pip install -r requirements.txt

In [ ]:
import geopandas as gpd
from rt_stop_times import * 
from shared_utils import gtfs_utils_v2, rt_dates

In [ ]:
TARGET_DATE = rt_dates.DATES["feb2025"]

In [ ]:
rt_schedule_stop_times = assemble_scheduled_rt_stop_times(
    TARGET_DATE,
    [*GTFS_DATA_DICT.rt_stop_times.trip_stop_cols]
)

### Get an example trip with non-monotonic stop_sequence values

In [ ]:
example_trip = rt_schedule_stop_times.loc[
    (rt_schedule_stop_times.schedule_gtfs_dataset_key == "c65bd95ac0009a74df9ff840fc416771")
    & (rt_schedule_stop_times.trip_id == "902110")
].sort_values("stop_sequence")
example_trip["rt_non_monotonic"] = (
    example_trip["rt_arrival_sec"].shift(1) > example_trip["rt_arrival_sec"]
)
example_trip[["stop_sequence", "scheduled_arrival_sec", "rt_arrival_sec", "rt_non_monotonic"]]

### Get a list of agencies that have trips with rt times and not scheduled times

In [ ]:
agencies_with_nonscheduled_service = rt_schedule_stop_times.loc[
    
    (rt_schedule_stop_times.scheduled_arrival_sec.isna())
    & ~(rt_schedule_stop_times.rt_arrival_sec.isna())
].schedule_gtfs_dataset_key.unique()

In [ ]:
agencies_with_nonscheduled_service

In [ ]:
gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(
        selected_date=TARGET_DATE, keep_cols=["name", "gtfs_dataset_key"]
).set_index("gtfs_dataset_key").loc[agencies_with_nonscheduled_service]